In [4]:
import jax
import jax.numpy as jnp
import chex
from jax.scipy.stats import norm
from annealed_flow_transport.utils import smc_utils as su
import matplotlib.pyplot as plt
from annealed_flow_transport.utils.aft_types import StepSizeSchedule, InterpolatedStepSizeSchedule

In [5]:
key = jax.random.PRNGKey(5)
key, key_ = jax.random.split(key)

In [6]:
x = InterpolatedStepSizeSchedule(1, 2, 3)
isinstance(x, StepSizeSchedule)

True